In [1]:
# !pip install -U torch==2.3.0 -q
# !pip install -U torchtext==0.18.0 -q

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/operations/check.py", line 101, in check_install_conflicts
    package_set, _ = create_package_set_from_installed()
              

In [43]:
import pandas as pd
import torch.nn as nn
import torch
from torchtext.transforms import ToTensor
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
import string # for splitting (tokenisation)

In [17]:
train_data = pd.read_csv('/content/twitter_training.csv')
train_data.head()

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [18]:
# missing data?
train_data.isnull().sum()

,0
2401,0
Borderlands,0
Positive,0
"im getting on borderlands and i will murder you all ,",686


In [19]:
# dropping unknown data
train_data = train_data.dropna()

In [26]:
len(train_data)

73995

Keeping only two columns that seem to be necessary: reviews and tweets, but also will just change the column names

In [20]:
train_data = train_data.drop(columns=['2401', 'Borderlands'])

# renaming the useful columns
train_data = train_data.rename(columns={'Positive': 'Sentiment', 'im getting on borderlands and i will murder you all ,': 'Text'})
train_data.head()

,Sentiment,Text
0,Positive,I am coming to the borders and I will kill you...
1,Positive,im getting on borderlands and i will kill you ...
2,Positive,im coming on borderlands and i will murder you...
3,Positive,im getting on borderlands 2 and i will murder ...
4,Positive,im getting into borderlands and i can murder y...


Neat, next step is to prepare the data in forms that will be ready to be fed in to the neural network

In [21]:
label_encoder = LabelEncoder()
train_data['Sentiment'] = label_encoder.fit_transform(train_data['Sentiment'])
train_data.head()

,Sentiment,Text
0,3,I am coming to the borders and I will kill you...
1,3,im getting on borderlands and i will kill you ...
2,3,im coming on borderlands and i will murder you...
3,3,im getting on borderlands 2 and i will murder ...
4,3,im getting into borderlands and i can murder y...


I am seeing 3, how many sentiment groups are there?

In [22]:
train_data["Sentiment"].value_counts()

,count
Sentiment,
1,22358
3,20654
2,18108
0,12875


In [23]:
# What then is their actual label?
label_encoder.classes_

array(['Irrelevant', 'Negative', 'Neutral', 'Positive'], dtype=object)

Interesting... Now to prepare the dataset

In [24]:
# prepare label and tweet fields

# convert to numpy array
train_np = train_data.to_numpy()

In [27]:
train_np[2]

array([3, 'im coming on borderlands and i will murder you all,'],
      dtype=object)

In [25]:
# build vocabs
tokenizer = lambda x: x.split()

def yield_tokens(data):
    for _, text in data:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_np), specials=["<unk>", "<pad>"])
vocab.set_default_index(vocab["<unk>"])

In [34]:
# transform dataset from built vocab

def text_transform(x):
    tokens = tokenizer(x)
    token_ids = vocab(tokens)
    return torch.tensor(token_ids, dtype=torch.long)

label_transform = lambda x: torch.tensor(x, dtype=torch.float)

# custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label, text = self.data[idx]
        return text_transform(text), label_transform(label)

# dataset
dataset = CustomDataset(train_np)

dataset[0]

(tensor([    4,   102,   436,     3,     2, 45838,     5,     4,    48,   522,
            18,  2186]),
 tensor(3.))

Now to batching the dataset into a dataloader

In [39]:
# dataloader

# collate_fn
def custom_collate_fn(batch):
    # extract text tensors and label
    text_tensors, labels = zip(*batch)
    # Stack labels
    labels = torch.stack(labels)
    # padding
    padded_texts = pad_sequence(text_tensors, batch_first=True, padding_value=vocab['<pad>'])

    # Return padded text and stacked labels
    return padded_texts, labels

# dataloader, using the custom collate function
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=custom_collate_fn)

# what does a single batch look like?
for batch in dataloader:
    print(batch)
    break

(tensor([[11267,   922,  4319,  ...,     1,     1,     1],
        [    4,   102,   217,  ...,     1,     1,     1],
        [ 1628,    82,     3,  ...,     1,     1,     1],
        ...,
        [  243,    18,   191,  ...,     1,     1,     1],
        [ 3898,   141,   148,  ...,     1,     1,     1],
        [ 3940,    27,    16,  ...,     1,     1,     1]]), tensor([1., 3., 0., 2., 2., 3., 2., 1., 2., 3., 1., 2., 1., 3., 0., 1., 1., 1.,
        1., 2., 2., 3., 3., 2., 2., 3., 3., 3., 3., 1., 1., 0.]))


Looks like we can proceed to the model struture and the training loop now

In [42]:
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, num_layers, bidirectional, dropout):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=vocab['<pad>'])
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, bidirectional=bidirectional, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):
        embedded = self.dropout(self.embedding(text))
        output, (hidden, cell) = self.lstm(embedded) # hidden is only needed here, the rest are just use when we need to keep the entire context in tasks like translation and text generation
        if self.lstm.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
        return self.fc(hidden)

vocab_size = len(vocab)
embedding_dim = 100
hidden_dim = 256
output_dim = 4
num_layers = 2
bidirectional = True
dropout = 0.5

model = LSTMModel(vocab_size, embedding_dim, hidden_dim, output_dim, num_layers, bidirectional, dropout)


In [45]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss() # uses softmax, so suitable for multiclass
optimizer = optim.Adam(model.parameters())

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    for batch in dataloader:
        text, labels = batch
        text = text.to(device)
        labels = labels.to(device).long() # Convert labels to Long for CrossEntropyLoss

        optimizer.zero_grad()
        predictions = model(text)
        loss = criterion(predictions, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print("Training finished.")


Epoch [1/20], Loss: 0.6096
Epoch [2/20], Loss: 1.0118
Epoch [3/20], Loss: 0.1261
Epoch [4/20], Loss: 0.4631
Epoch [5/20], Loss: 0.2416
Epoch [6/20], Loss: 1.1215
Epoch [7/20], Loss: 0.3253
Epoch [8/20], Loss: 0.1683
Epoch [9/20], Loss: 0.2815
Epoch [10/20], Loss: 0.0860
Epoch [11/20], Loss: 0.4679
Epoch [12/20], Loss: 0.0492
Epoch [13/20], Loss: 0.3635
Epoch [14/20], Loss: 0.3678
Epoch [15/20], Loss: 0.1554
Epoch [16/20], Loss: 0.0751
Epoch [17/20], Loss: 0.6665
Epoch [18/20], Loss: 0.3116
Epoch [19/20], Loss: 0.0533
Epoch [20/20], Loss: 0.0941
Training finished.


Testing with the validation data

In [48]:
val_data = pd.read_csv('/content/twitter_validation.csv')
val_data.head()

,3364,Facebook,Irrelevant,"I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣"
0,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
1,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
2,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
3,4433,Google,Neutral,Now the President is slapping Americans in the...
4,6273,FIFA,Negative,Hi @EAHelp I’ve had Madeleine McCann in my cel...


In [49]:
val_data.drop(columns=['3364', 'Facebook'], inplace=True)
val_data = val_data.rename(columns={'Irrelevant': 'Sentiment',
                                    'I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣': 'Text'})
val_data.head()

,Sentiment,Text
0,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
1,Negative,@Microsoft Why do I pay for WORD when it funct...
2,Negative,"CSGO matchmaking is so full of closet hacking,..."
3,Neutral,Now the President is slapping Americans in the...
4,Negative,Hi @EAHelp I’ve had Madeleine McCann in my cel...


In [51]:
val_data.dropna(inplace=True)
val_data["Sentiment"].value_counts()

,count
Sentiment,
Neutral,285
Positive,277
Negative,266
Irrelevant,171


In [52]:
val_data['Sentiment'] = label_encoder.transform(val_data['Sentiment'])
val_data.head()

,Sentiment,Text
0,2,BBC News - Amazon boss Jeff Bezos rejects clai...
1,1,@Microsoft Why do I pay for WORD when it funct...
2,1,"CSGO matchmaking is so full of closet hacking,..."
3,2,Now the President is slapping Americans in the...
4,1,Hi @EAHelp I’ve had Madeleine McCann in my cel...


In [53]:
val_np = val_data.to_numpy()
val_dataset = CustomDataset(val_np)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=custom_collate_fn)

In [54]:
def test_model(model, dataloader, device):
    model.eval()
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for batch in val_dataloader:
            text, labels = batch
            text = text.to(device)
            labels = labels.to(device).long()

            predictions = model(text)
            _, predicted_labels = torch.max(predictions, 1)

            total_samples += labels.size(0)
            correct_predictions += (predicted_labels == labels).sum().item()

    accuracy = (correct_predictions / total_samples) * 100
    print(f"Test Accuracy: {accuracy:.2f}%")

test_model(model, val_dataloader, device)

Test Accuracy: 96.50%


And, that is neat

In [57]:
torch.save(model,"/content/senti-model.pt")

In [58]:
import joblib

# Save the required elements using joblib
joblib.dump(vocab, '/content/vocab.joblib')
joblib.dump(label_encoder, '/content/label_encoder.joblib')


['/content/label_encoder.joblib']